In [1]:
# RODOR algorithm for coin counting, procesor
# Based on paper RODOR algorithm for the identification of irregular surfaces with circular 
# contours applied to the recognition and counting of coins
# Paper and code by Msc Evelyn Orellana and Msc Edgar Rodriguez. 2023
# Paper on https://www.revistatoolbar.com/
# Email :    eorellana@revistatoolbar.com,     erodriguez@revistatoolbar.com
#
# This project is licensed under the terms of the MIT License

from RCC_Modules import *

In [2]:
def RCCprocess(DataFile, ImageFile, ObjCoinImages):
    print('Begin process segmentation for coin counting ', time.strftime("%d/%m/%y %H:%M:%S") )
    vCurrency = pd.read_csv( OutputDir + DataFile )
    
    CoinImages = []
    CCircles, ImageColor, ImageGray = fnFindCoins(ImageFile)
    
    for CC in CCircles:
        t0 = time.time()
        DiameterCC, xLTC, yLTC = CC   # x, y in Left top corner ( LTC ) of image
        vcCoin  = vCurrency
        
        ImageGraySec = ImageGray[yLTC-3:yLTC+DiameterCC+3, xLTC-3:xLTC+DiameterCC+3]
        Diameter, Radio, SegmentEdge, SegmentSunken = fnImageProcess(ImageColor, ImageGraySec, 'Process',ImageFile+'GraySec x'+str(xLTC) +' y'+ str(yLTC) )
        
        vcCoin['DiameterAcc'] = vcCoin['Diameter'].apply(fnAccuracy, args = [Diameter])
        vcCoin['RadioAcc'] = vcCoin['Radio'].apply(fnAccuracy, args = [Radio])
        vcCoin['SegmentEdgeAcc'] = vcCoin['SegmentEdge'].apply(fnAccuracy, args = [SegmentEdge])
        vcCoin['SegmentSunkenAcc'] = vcCoin['SegmentSunken'].apply(fnAccuracy, args = [SegmentSunken])
        vcCoin['AvgAccuracy'] = (vcCoin['RadioAcc'] + vcCoin['SegmentEdgeAcc'] + vcCoin['SegmentSunkenAcc'] + vcCoin['DiameterAcc' ]) / 4

        vcCoin = vcCoin.set_index('Id')
        idx = vcCoin['AvgAccuracy'].idxmax()        
        if( int(vcCoin['AvgAccuracy'][idx]) >= Accuracy ):
            t1 = time.time()
            et = "{:.3f}s".format(t1-t0)
            CoinImages.append([vcCoin['AvgAccuracy'][idx],Diameter, xLTC, yLTC,vcCoin['Country'][idx],vcCoin['Currency'][idx],vcCoin['CurrencyValue'][idx],et]) 
            
    with open( OutputDir + ImageFile + ObjCoinImages, 'wb') as fileobj:
        pickle.dump(CoinImages, fileobj )
    
    print('End process segmentation for coin counting ', time.strftime("%d/%m/%y %H:%M:%S") )    

    return CoinImages

In [3]:
CoinImages = RCCprocess(DataFile='DataCurrency.csv', ImageFile='CoinsALL-dp2', ObjCoinImages = '_obj.pickle' )

print('Images found\n',CoinImages)

Begin process segmentation for coin counting  16/12/23 12:03:02
End process segmentation for coin counting  16/12/23 12:03:27
Images found
 [[99.62571463625054, 80, 599, 31, 'FR', 'EUR', 0.05, '0.146s'], [99.46635389425349, 90, 242, 33, 'GT', 'GTQ', 0.5, '0.198s'], [98.24572034657164, 70, 152, 34, 'GT', 'GTQ', 0.01, '0.097s'], [99.51516040622224, 74, 424, 35, 'ES', 'EUR', 0.1, '0.098s'], [97.81324801817621, 96, 778, 35, 'ES', 'EUR', 2.0, '0.150s'], [99.63863337713535, 60, 908, 35, 'ES', 'EUR', 0.01, '0.068s'], [99.3449395376555, 80, 1677, 36, 'FR', 'EUR', 0.05, '0.109s'], [99.58362484270965, 90, 1320, 38, 'GT', 'GTQ', 0.5, '0.132s'], [98.27936865735329, 70, 1230, 39, 'GT', 'GTQ', 0.01, '0.088s'], [99.88579766402943, 74, 1004, 40, 'ES', 'EUR', 0.1, '0.107s'], [99.39867387415416, 74, 1502, 40, 'ES', 'EUR', 0.1, '0.103s'], [96.87708961696924, 94, 1856, 40, 'ES', 'EUR', 2.0, '0.147s'], [99.50205556294684, 60, 1986, 40, 'ES', 'EUR', 0.01, '0.069s'], [99.79900620872112, 74, 2082, 45, 'ES', '